## Suffering niggas

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh.plotting
import catboost
import sklearn.preprocessing, sklearn.feature_selection, sklearn.model_selection
from sklearn.pipeline import Pipeline, make_pipeline
import sklearn.base
from sklearn.base import BaseEstimator, TransformerMixin

SEED = 42
np.random.seed = SEED

%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
display(
    X_train.shape,
    X_train.columns,
    X_train.isnull().sum(),
)

(59400, 40)

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [4]:
pd.Series({c: ([X_train[c].dtype] + list(X_train[c].unique()[:4])).__repr__() for c in X_train.columns})

amount_tsh                     [dtype('float64'), 6000.0, 0.0, 25.0, 20.0]
basin                    [dtype('O'), 'Lake Nyasa', 'Lake Victoria', 'P...
construction_year                 [dtype('int64'), 1999, 2010, 2009, 1986]
date_recorded            [dtype('O'), '2011-03-14', '2013-03-06', '2013...
district_code                                [dtype('int64'), 5, 2, 4, 63]
extraction_type          [dtype('O'), 'gravity', 'submersible', 'swn 80...
extraction_type_class    [dtype('O'), 'gravity', 'submersible', 'handpu...
extraction_type_group    [dtype('O'), 'gravity', 'submersible', 'swn 80...
funder                   [dtype('O'), 'Roman', 'Grumeti', 'Lottery Club...
gps_height                          [dtype('int64'), 1390, 1399, 686, 263]
id                             [dtype('int64'), 69572, 8776, 34310, 67743]
installer                [dtype('O'), 'Roman', 'GRUMETI', 'World vision...
latitude                 [dtype('float64'), -9.8563217699999992, -2.147...
lga                      

In [5]:
y_train.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [6]:
import dateutil.parser

In [7]:
class HideMissingValues(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for c in X.columns:
            if np.issubdtype(X[c].dtype, np.number):
                X.loc[:,c].fillna(X.loc[:,c].median(), inplace=True)
            else:
                X.loc[:,c].fillna('Unknown', inplace=True)
                X.loc[:,c] = X[c].map(lambda x: str(x))
        return X
    
    
class DropGarbage(BaseEstimator, TransformerMixin):
    def __init__(self, cols=['id', 'funder', 'recorded_by', 'date_recorded']):
        self.cols = cols
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(self.cols, axis=1)

class SplitDate(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        def t(x):
            if isinstance(x, str):
                x = dateutil.parser.parse(x)
            return (x.year, x.month, x.day, x.weekday())
        X = X.copy()
        (X.loc[:, self.col + '_year'],
         X.loc[:, self.col + '_month'],
         X.loc[:, self.col + '_day'],
         X.loc[:, self.col + '_weekday']) = zip(
        *X.loc[:, self.col].map(t))
        return X

In [8]:
from sklearn.model_selection import train_test_split

In [11]:
prepr1 = Pipeline([
    ('fillna', HideMissingValues()),
    ('date_recorded', SplitDate('date_recorded')),
    ('drop', DropGarbage(cols=['id', 'recorded_by', ]))
])

y_enc = sklearn.preprocessing.LabelEncoder()
X = prepr1.fit_transform(X_train)
y = y_enc.fit_transform(y_train.iloc[:,1])

X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.66)

!date
cbc1 = catboost.CatBoostClassifier(random_seed=SEED,
         loss_function='MultiClass',
         eval_set=(X_te, y_te))
cbc1.fit(X_tr,
         y_tr,
         cat_features=np.where(np.array([dt == object for dt in X.dtypes]))[0],
         verbose=False)
!date

Thu Jul 27 18:12:14 MSK 2017
Thu Jul 27 18:25:23 MSK 2017


In [12]:
cbc1.score(X_te, y_te)

0.80842741136858787

In [ ]:
!head SubmissionFormat.csv

In [15]:
y_test1 = cbc1.predict(prepr1.transform(X_test))

In [16]:
y_test1[:4]

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 2.]])

In [25]:
ans1 = pd.DataFrame({'status_group': y_enc.inverse_transform(y_test1.astype(int).ravel())},
                    index= X_test.iloc[:,0].values)
ans1.index.name = 'id'
ans1.to_csv('ans1.csv')

In [26]:
!head ans1.csv

id,status_group
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional
52449,functional
24806,functional
28965,non functional
36301,non functional


In [13]:
class HandyFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        X.loc[:,'amount_per_man'] = X.loc[:,'amount_tsh']/X.loc[:,'population']
        return X

In [14]:
prepr2 = Pipeline([
    ('fillna', HideMissingValues()),
    ('date_recorded', SplitDate('date_recorded')),
    ('handyfeats', HandyFeatures()),
    ('drop', DropGarbage(cols=['id', 'recorded_by']))
])

y_enc = sklearn.preprocessing.LabelEncoder()
X = prepr2.fit_transform(X_train)
y = y_enc.fit_transform(y_train.iloc[:,1])

X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.66)

!date
cbc2 = catboost.CatBoostClassifier(random_seed=SEED,
         loss_function='MultiClass',
         eval_set=(X_te, y_te))
cbc2.fit(X_tr,
         y_tr,
         cat_features=np.where(np.array([dt == object for dt in X.dtypes]))[0],
         verbose=False)
!date
display('score: %s' % cbc2.score(X_te, y_te))
y_test2 = cbc2.predict(prepr2.transform(X_test))
ans2 = pd.DataFrame({'status_group': y_enc.inverse_transform(y_test2.astype(int).ravel())},
                    index= X_test.iloc[:,0].values)
ans2.index.name = 'id'
ans2.to_csv('ans2.csv')

Thu Jul 27 20:46:30 MSK 2017
Thu Jul 27 21:00:41 MSK 2017


'score: 0.806001188354'